In [2]:
%load_ext dotenv
%dotenv

import os;
import requests;
import pandas as pd;
import numpy as np;

In [3]:
# This section permits a lazy way to retrieve your personal inputs.
# To make it work, log into AoC from your browser and go into developer options to look at your cookies
# I know this works for logging in via github, but I don't know if it works for other auth styles.
# Note the following cookie values: _ga, _gid, and session.
#
# Create a file in the root of this project folder called ".env" and in it, put the following:
# AOC_GA=9243572943857293478529084
# AOC_GID=004785928437508347502384534
# AOC_SESSION=840372b93dj387023n729c5304857fj20f
# where the values match your actual login params.

def get_inputs(day):
    url="https://adventofcode.com/2019/day/%d/input" % day
    
    cookies = {
#         '_ga': os.getenv("AOC_GA"),
#         '_gid': os.getenv("AOC_GID"),
        'session': os.getenv("AOC_SESSION")
    }
    return requests.get(url,cookies=cookies).text.split()

In [ ]:
# Day 1

if 'inputs_1' not in locals():
    inputs_1 = get_inputs(1)

def get_fuel(mass):
    fuel = mass/3
    fuel = int(fuel) - 2
    return fuel
    
def test_get_fuel(mass,fuel):
    tested_fuel = get_fuel(mass)
    assert tested_fuel == fuel, "Fuel for %d should be %d but was %d" % (mass, fuel, tested_fuel)

def test_part1():
    test_get_fuel(12,2)
    test_get_fuel(14,2)
    test_get_fuel(1969,654)
    test_get_fuel(100756,33583)
    print("Part 1 test successful")
    
def day1():
    test_part1();
    total_fuel = 0;
    for input in inputs_1:
        total_fuel = total_fuel + get_fuel(int(input))
        
    print(total_fuel)

# ------------------------------------------------------------------------------------------

def fuel_for_fuel(fuel, total_extra_fuel=0):
    extra_fuel = get_fuel(fuel)
    if extra_fuel <= 0:
        return total_extra_fuel
    else:
        total_extra_fuel += extra_fuel
        return fuel_for_fuel(extra_fuel,total_extra_fuel)

def test_fuel_for_fuel(mass,fuel):
    tested_fuel = get_fuel(mass)
    tested_fuel += fuel_for_fuel(tested_fuel)
    assert tested_fuel == fuel, "Fuel for %d should be %d but was %d" % (mass, fuel, tested_fuel)

def test_part2():
    test_fuel_for_fuel(14,2)
    test_fuel_for_fuel(1969,966)
    test_fuel_for_fuel(100756,50346)
    print("Part 2 test successful")

def day2():
    test_part2();
    total_fuel = 0;
    for input in inputs_1:
        fuel = get_fuel(int(input))
        total_fuel = total_fuel + fuel + fuel_for_fuel(fuel)
        
    print(total_fuel)

day1()
day2()

In [19]:
# Day 2

if 'inputs_2' not in locals():
    inputs_2 = get_inputs(2)[0].split(',')
    inputs_2 = list(map(lambda num: int(num),inputs_2))
    
test_inputs_2 = [
    {"test_in":[1,9,10,3,2,3,11,0,99,30,40,50], "test_out":[3500,9,10,70,2,3,11,0,99,30,40,50]},
    {"test_in":[1,0,0,0,99],                    "test_out":[2,0,0,0,99]},
    {"test_in":[2,3,0,3,99],                    "test_out":[2,3,0,6,99]},
    {"test_in":[2,4,4,5,99,0],                  "test_out":[2,4,4,5,99,9801]},
    {"test_in":[1,1,1,4,99,5,6,0,99],           "test_out":[30,1,1,4,2,5,6,0,99]},
]

"""
Return False if finished
"""
def process(pos, inputs):
    if( pos >= len(inputs)):
        raise Exception("Asked for pos %d but length of input is %d" % (pos,len(inputs)))

#     print("Processing %d: %d - %s" % (pos, inputs[pos], inputs))    
    
    if( inputs[pos] == 1):
        target_pos = inputs[pos+3]
        val1_pos = inputs[pos+1]
        val2_pos = inputs[pos+2]
#         print("  Changing %d from %d to %d" % (target_pos, inputs[target_pos], inputs[val1_pos]+inputs[val2_pos] ))
        inputs[target_pos] = inputs[val1_pos] + inputs[val2_pos]
        return True
    elif( inputs[pos] == 2):
        target_pos = inputs[pos+3]
        val1_pos = inputs[pos+1]
        val2_pos = inputs[pos+2]
#         print("  Changing %d from %d to %d" % (target_pos, inputs[target_pos], inputs[val1_pos]*inputs[val2_pos] ))
        inputs[target_pos] = inputs[val1_pos] * inputs[val2_pos]
        return True
    elif( inputs[pos] == 99):
#         del inputs[pos+1:]
        return False
    else:
        raise Exception("Opcode at position %d was %d (out of spec)" % (pos,inputs[pos]))
    
def process_all(inputs):
    inputs_result = inputs.copy()
    
    pos = 0;
    still_running = process(pos,inputs_result)
    
    while still_running:
        pos += 4
        still_running = process(pos,inputs_result)
        
#     print( "Done: %s" % inputs_result )
    return inputs_result
    
def test_day2_part1():
    for test_input_2 in test_inputs_2:
        test_result = process_all(test_input_2["test_in"])
        assert len(test_result) == len(test_input_2["test_out"]), "Different array lens: %s, %s" % (test_result,test_input_2["test_out"])
        for i in range(0,len(test_input_2["test_out"])):
            assert test_result[i] == test_input_2["test_out"][i], "Pos %d should be %d but was %d" % (i,test_input_2["test_out"][i],test_result[i])

def day1():
    working_array = inputs_2.copy()
    working_array[1] = 12
    working_array[2] = 2
    result = process_all(working_array)
    print("Array at position 0 is %d" % result[0])
    
            
test_day2_part1()
day1()
        

Array at position 0 is 3895705
